In [1]:
import pandas as pd
import requests

In [2]:
# Query url for station info of City Bike
url = "https://gbfs.citibikenyc.com/gbfs/en/station_information.json"

# Get data of City Bike station info and save as "sta_info"
sta_info_url = requests.get(url).json()["data"]["stations"]

# View "sta_info"
sta_info_url[0]

{'station_id': '298',
 'external_id': '66db69d0-0aca-11e7-82f6-3863bb44ef7c',
 'name': '3 Ave & Schermerhorn St',
 'short_name': '4437.01',
 'lat': 40.68683208,
 'lon': -73.9796772,
 'region_id': 71,
 'rental_methods': ['CREDITCARD', 'KEY'],
 'capacity': 35,
 'rental_url': 'http://app.citibikenyc.com/S6Lr/IBV092JufD?station_id=298',
 'electric_bike_surcharge_waiver': False,
 'eightd_has_key_dispenser': False,
 'eightd_station_services': [{'id': '5387339b-dcb0-4070-8e51-4fb719d3d482',
   'service_type': 'ATTENDED_SERVICE',
   'bikes_availability': 'UNLIMITED',
   'docks_availability': 'NONE',
   'name': 'Valet Service',
   'description': 'Citibike Valet Attendant Service Available',
   'schedule_description': '',
   'link_for_more_info': 'https://www.citibikenyc.com/valet'}],
 'has_kiosk': True}

In [3]:
# Simplify station info to be saved as Pandas dataframe
# Note that "-" is used for conjunction as Tableau recognize "_" as space " "
sta_info_df = pd.DataFrame(columns = ["s-sta-id", "name", "lat", "lng", "region-id"])

# Loop through "sta_info_url"
for sta in sta_info_url:

    # Note that adding value to any column MIGHT change the length of "sta_info_df" even within the same iteration
    # Use "im" (intermediate) to store the length of "sta_info_df" for each iteration
    im = len(sta_info_df.index)
    
    # Append value from "sta_info_url" to its designated column in "sta_info_df"
    sta_info_df.loc[im, "s-sta-id"] = sta["station_id"]   # "s-sta-id" for inner join with start station id
    sta_info_df.loc[im, "name"] = sta["name"]
    sta_info_df.loc[im, "lat"] = sta["lat"]
    sta_info_df.loc[im, "lng"] = sta["lon"]
    sta_info_df.loc[im, "region-id"] = sta["region_id"]

# Preview "sta_info_pd"    
sta_info_df.head()

,s-sta-id,name,lat,lng,region-id
0,298,3 Ave & Schermerhorn St,40.6868,-73.9797,71
1,307,Canal St & Rutgers St,40.7143,-73.9899,71
2,312,Allen St & Stanton St,40.7221,-73.9891,71
3,358,Christopher St & Greenwich St,40.7329,-74.0071,71
4,426,West St & Chambers St,40.7175,-74.0132,71


In [4]:
# Create another column "e-sta-id" for inner join with end station id
sta_info_df["e-sta-id"] = sta_info_df["s-sta-id"]

# Preview "sta_info_df"
sta_info_df.head()

,s-sta-id,name,lat,lng,region-id,e-sta-id
0,298,3 Ave & Schermerhorn St,40.6868,-73.9797,71,298
1,307,Canal St & Rutgers St,40.7143,-73.9899,71,307
2,312,Allen St & Stanton St,40.7221,-73.9891,71,312
3,358,Christopher St & Greenwich St,40.7329,-74.0071,71,358
4,426,West St & Chambers St,40.7175,-74.0132,71,426


In [5]:
# Reorganize the columns of "sta_info_df"
sta_info_df = sta_info_df[["s-sta-id", "e-sta-id", "name", "lat", "lng", "region-id"]]

# Preview "sta_info_df"
sta_info_df.head()

,s-sta-id,e-sta-id,name,lat,lng,region-id
0,298,298,3 Ave & Schermerhorn St,40.6868,-73.9797,71
1,307,307,Canal St & Rutgers St,40.7143,-73.9899,71
2,312,312,Allen St & Stanton St,40.7221,-73.9891,71
3,358,358,Christopher St & Greenwich St,40.7329,-74.0071,71
4,426,426,West St & Chambers St,40.7175,-74.0132,71


In [6]:
# Save sta_info as csv file
sta_info_df.to_csv("../data/cleaned_data/station_info.csv", index=False, header=True)